In [1]:
# -*- coding: utf-8 -*-

import sys,os
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import linalg
from numpy import dot
import geomloss as gs

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as D
from torchvision import datasets, transforms
import torchvision.utils as vutils
from torch.autograd import grad
import torch.utils.data
import torch.backends.cudnn as cudnn
from torch.nn.modules import Linear
from torch.autograd.functional import jacobian,hessian,vjp,vhp,hvp

import random
import math

FilePath = '../../'

file_list = ['GSM1599494_ES_d0_main.csv', 'GSM1599497_ES_d2_LIFminus.csv', 'GSM1599498_ES_d4_LIFminus.csv', 'GSM1599499_ES_d7_LIFminus.csv']

table_list = []
for filein in file_list:
    table_list.append(pd.read_csv(FilePath+filein, header=None))

matrix_list = []
gene_names = table_list[0].values[:,0]
for table in table_list:
    matrix_list.append(table.values[:,1:].astype('float32'))

cell_counts = [matrix.shape[1] for matrix in matrix_list]

def normalize_run(mat):
    rpm = np.sum(mat,0)/1e6
    detect_pr = np.sum(mat==0,0)/float(mat.shape[0])
    return np.log(mat*(np.median(detect_pr)/detect_pr)*1.0/rpm + 1.0)

norm_mat = [normalize_run(matrix) for matrix in matrix_list]

qt_mat = [np.percentile(norm_in,q=np.linspace(0,100,50),axis=1) for norm_in in norm_mat] 
wdiv=np.sum((qt_mat[0]-qt_mat[3])**2,0)
w_order = np.argsort(-wdiv)

wsub = w_order[0:100]


def nmf(X, latent_features, max_iter=100, error_limit=1e-6, fit_error_limit=1e-6, print_iter=200):
    """
    Decompose X to A*Y
    """
    eps = 1e-5
    print('Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter))
    #X = X.toarray()   I am passing in a scipy sparse matrix

    # mask
    mask = np.sign(X)

    # initial matrices. A is random [0,1] and Y is A\X.
    rows, columns = X.shape
    A = np.random.rand(rows, latent_features)
    A = np.maximum(A, eps)

    Y = linalg.lstsq(A, X)[0]
    Y = np.maximum(Y, eps)

    masked_X = mask * X
    X_est_prev = dot(A, Y)
    for i in range(1, max_iter + 1):
        # ===== updates =====
        # Matlab: A=A.*(((W.*X)*Y')./((W.*(A*Y))*Y'));
        top = dot(masked_X, Y.T)
        bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
        A *= top / bottom

        A = np.maximum(A, eps)
        # print 'A',  np.round(A, 2)

        # Matlab: Y=Y.*((A'*(W.*X))./(A'*(W.*(A*Y))));
        top = dot(A.T, masked_X)
        bottom = dot(A.T, mask * dot(A, Y)) + eps
        Y *= top / bottom
        Y = np.maximum(Y, eps)
        # print 'Y', np.round(Y, 2)


        # ==== evaluation ====
        if i % print_iter == 0 or i == 1 or i == max_iter:
            print('Iteration {}:'.format(i),)
            X_est = dot(A, Y)
            err = mask * (X_est_prev - X_est)
            fit_residual = np.sqrt(np.sum(err ** 2))
            X_est_prev = X_est

            curRes = linalg.norm(mask * (X - X_est), ord='fro')
            print('fit residual', np.round(fit_residual, 4),)
            print('total residual', np.round(curRes, 4))
            if curRes < error_limit or fit_residual < fit_error_limit:
                break
    return A, Y, dot(A,Y)

np.random.seed(0)
norm_imputed = [nmf(normin[wsub,:], latent_features = len(wsub)*4, max_iter=500)[2] for normin in norm_mat]

norm_adj = np.mean(norm_imputed[3],1)[:,np.newaxis]
subvec = np.array([0,1,2,3,4,5,6,7,8,9])

gnvec = gene_names[w_order[subvec]]

cov_mat = np.cov(norm_imputed[3][subvec,:])
whiten = np.diag(np.diag(cov_mat)**(-0.5))
unwhiten = np.diag(np.diag(cov_mat)**(0.5))

norm_imputed2 = [np.dot(whiten,(normin - norm_adj)[subvec,:]) for normin in norm_imputed]


class MLP(nn.Module):

    def __init__(self, dim_in, dim_out, dim_hidden=64, num_hidden=0, activation=nn.LeakyReLU()):
        super(MLP, self).__init__()

        if num_hidden == 0:
            self.linears = nn.ModuleList([nn.Linear(dim_in, dim_out)])
        elif num_hidden >= 1:
            self.linears = nn.ModuleList() 
            self.linears.append(nn.Linear(dim_in, dim_hidden))
            self.linears.extend([nn.Linear(dim_hidden, dim_hidden) for _ in range(num_hidden-1)])
            self.linears.append(nn.Linear(dim_hidden, dim_out))
        else:
            raise Exception('number of hidden layers must be positive')

        for m in self.linears:
            #nn.init.xavier_uniform_(m.weight)
            nn.init.xavier_normal_(m.weight)
            nn.init.uniform_(m.bias,a=-0.1,b=0.1)
            #nn.init.constant_(m.bias,0)
 
        self.activation = activation # 

    def forward(self, x):
        for m in self.linears[:-1]:
            x = self.activation(m(x))
            #x = F.dropout(x,p=0.5)

        return self.linears[-1](x)


def compute_gradient_penalty(D, real_sample, fake_sample,k,p):
    real_samples = real_sample.requires_grad_(True)
    fake_samples = fake_sample.requires_grad_(True)

    real_validity = D(real_samples)
    fake_validity = D(fake_samples)

    real_grad_out = torch.ones((real_samples.shape[0],1),dtype=torch.float32,requires_grad=False,device="cuda")
    real_grad = grad(
        real_validity, real_samples, real_grad_out, create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    real_grad_norm = real_grad.view(real_grad.size(0), -1).pow(2).sum(1) ** (p / 2)

    fake_grad_out = torch.ones((fake_samples.shape[0],1),dtype=torch.float32,requires_grad=False,device="cuda")
    fake_grad = grad(
        fake_validity, fake_samples, fake_grad_out, create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    fake_grad_norm = fake_grad.view(fake_grad.size(0), -1).pow(2).sum(1) ** (p / 2)

    return (torch.sum(real_grad_norm) + torch.sum(fake_grad_norm)) * k / (real_sample.shape[0]+fake_sample.shape[0])

class JumpEulerForwardCuda(nn.Module):
    def __init__(self,in_features,num_hidden,dim_hidden,step_size):
        super(JumpEulerForwardCuda,self).__init__()

        self.drift = MLP(in_features,in_features,dim_hidden,num_hidden)
        self.intensity = torch.tensor(intensity,device="cuda")
        self.mean = nn.Parameter(0.01*torch.ones(in_features))
        self.covHalf = nn.Parameter(0.08*torch.eye(in_features))
        self.diffusion = nn.Parameter(torch.ones(bd,10))
        self.in_features = in_features
        self.jump = MLP(in_features,in_features,dim_hidden,num_hidden)
        self.step_size = step_size

    def forward(self,z0,Nsim,steps):

        PopulationPath = torch.empty(size = (Nsim,steps+1,self.in_features),device="cuda")
        PopulationPath[:,0,:] = z0
        state = z0

        for i in range(1,steps+1):
            DP = D.poisson.Poisson(self.intensity*self.step_size) 
            pois = DP.sample((Nsim,1)).cuda()
            state = state + self.drift(state)*self.step_size + math.sqrt(self.step_size)*torch.normal(0,1,size=(Nsim,bd),device="cuda")@self.diffusion+\
                (pois*self.mean + pois**(0.5)*torch.normal(0,1,size=(Nsim,self.in_features),device="cuda")@self.covHalf)*self.jump(state)
            PopulationPath[:,i,:] = state
        return PopulationPath


def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
sed = 200
setup_seed(sed)

a=gs.SamplesLoss(loss='sinkhorn',p=2,blur=0.01)


train_data = norm_imputed2

train0 = torch.tensor(train_data[0],dtype=torch.float32,requires_grad = True,device="cuda").t()
train2 = torch.tensor(train_data[1],dtype=torch.float32,requires_grad = True,device="cuda").t()
train4 = torch.tensor(train_data[2],dtype=torch.float32,requires_grad = True,device="cuda").t()
train7 = torch.tensor(train_data[3],dtype=torch.float32,requires_grad = True,device="cuda").t()

train0 = train0+0.1*torch.normal(0,1,size=train0.shape,device="cuda")
train2 = train2+0.1*torch.normal(0,1,size=train2.shape,device="cuda")
train4 = train4+0.1*torch.normal(0,1,size=train4.shape,device="cuda")
train7 = train7+0.1*torch.normal(0,1,size=train7.shape,device="cuda")


intensity = 10
lr = 0.0003
step_size = 0.03
kuan = 256
ceng = 4
bd = 2
n_critic = 3
k = 2
p = 6

n_sims = train0.shape[0]
in_features = train0.shape[1]
n_steps = [10,20,35]


netG = JumpEulerForwardCuda(10,ceng,kuan,step_size).cuda()
netD1 = MLP(10,1,dim_hidden=kuan,num_hidden=ceng).cuda()
netD2 = MLP(10,1,dim_hidden=kuan,num_hidden=ceng).cuda()
netD3 = MLP(10,1,dim_hidden=kuan,num_hidden=ceng).cuda()


optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerSD1 = optim.Adam(netD1.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerSD2 = optim.Adam(netD2.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerSD3 = optim.Adam(netD3.parameters(), lr=lr, betas=(0.5, 0.999))

n_epochs =  20000

wd = []
for epoch in range(n_epochs):
  


    for _ in range(n_critic):
        fake_data = netG(train0,n_sims,n_steps[2])
        fake1 = fake_data[:,n_steps[0],:]
        fake2 = fake_data[:,n_steps[1],:]
        fake3 = fake_data[:,n_steps[2],:]

        optimizerSD1.zero_grad()

        div_gp1 = compute_gradient_penalty(netD1,train2,fake1,k,p)
        d1_loss = -torch.mean(netD1(train2))+torch.mean(netD1(fake1))+div_gp1
        d1_loss.backward(retain_graph=True) # retain_graph=True

        optimizerSD1.step()

        optimizerSD2.zero_grad()
        
        div_gp2 = compute_gradient_penalty(netD2,train4,fake2,k,p)
        d2_loss = -torch.mean(netD2(train4))+torch.mean(netD2(fake2))+div_gp2
        d2_loss.backward(retain_graph=True)

        optimizerSD2.step()
        
        
        optimizerSD3.zero_grad()
        
        div_gp3 = compute_gradient_penalty(netD3,train7,fake3,k,p)
        d3_loss = -torch.mean(netD3(train7))+torch.mean(netD3(fake3))+div_gp3
        d3_loss.backward(retain_graph=True)

        optimizerSD3.step()
        

    
    for _ in range(1):
        optimizerG.zero_grad()
        fake_data = netG(train0,n_sims,n_steps[2])
        fake1 = fake_data[:,n_steps[0],:]
        fake2 = fake_data[:,n_steps[1],:]
        fake3 = fake_data[:,n_steps[2],:]
        g_loss = -torch.mean(netD1(fake1))-torch.mean(netD2(fake2))-torch.mean(netD3(fake3))
        g_loss.backward() 

        optimizerG.step()

    if epoch %10==0:
        x1 = a(fake_data[:,n_steps[0],:],train2).item()
        x2 = a(fake_data[:,n_steps[1],:],train4).item()
        x3 = a(fake_data[:,n_steps[2],:],train7).item()
        
        wd.append(x1+x2+x3)
        
        print("training error: ",x1," and ",x2," and ",x3)
        print("total error: ",wd[-1])

Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
fit residual 4000.1233
total residual 138.7959
Iteration 200:
fit residual 123.6643
total residual 31.8464
Iteration 400:
fit residual 21.7894
total residual 12.6115
Iteration 500:
fit residual 4.3442
total residual 8.6686
Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
fit residual 2257.8358
total residual 58.2265
Iteration 200:
fit residual 56.8198
total residual 4.0447
Iteration 400:
fit residual 3.4777
total residual 0.6841
Iteration 500:
fit residual 0.3744
total residual 0.3185
Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
fit residual 2925.3664
total residual 80.2657
Iteration 200:
fit residual 78.1148
total residual 6.1559
Iteration 400:
fit residual 5.1661
total residual 1.1917
Iteration 500:
fit residual 0.6183
total residual 0.5917
Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
f

training error:  1.7011101245880127  and  2.2476353645324707  and  1.7428810596466064
total error:  5.69162654876709
training error:  1.721401333808899  and  2.3279826641082764  and  1.611617088317871
total error:  5.661001086235046
training error:  1.703049659729004  and  2.387953758239746  and  1.9327341318130493
total error:  6.023737549781799
training error:  1.6429194211959839  and  2.144483804702759  and  1.5250988006591797
total error:  5.312502026557922
training error:  1.6946673393249512  and  2.2536120414733887  and  1.5457133054733276
total error:  5.4939926862716675
training error:  1.6849839687347412  and  2.7425713539123535  and  2.1006228923797607
total error:  6.5281782150268555
training error:  1.845886468887329  and  3.0585598945617676  and  1.5953292846679688
total error:  6.499775648117065
training error:  1.7131321430206299  and  3.988844633102417  and  2.4839773178100586
total error:  8.185954093933105
training error:  1.8092446327209473  and  3.6468639373779297  

training error:  1.388885736465454  and  2.369910717010498  and  1.4723564386367798
total error:  5.231152892112732
training error:  1.323105812072754  and  1.9557515382766724  and  1.5739493370056152
total error:  4.8528066873550415
training error:  1.3234117031097412  and  2.0262324810028076  and  1.479233741760254
total error:  4.828877925872803
training error:  1.3494226932525635  and  1.9852122068405151  and  1.4553626775741577
total error:  4.789997577667236
training error:  1.3052940368652344  and  2.2015583515167236  and  1.590705156326294
total error:  5.097557544708252
training error:  1.3390984535217285  and  2.522245407104492  and  1.5170924663543701
total error:  5.378436326980591
training error:  1.3283052444458008  and  2.1996772289276123  and  1.6121844053268433
total error:  5.140166878700256
training error:  1.283340334892273  and  2.2637763023376465  and  1.5765883922576904
total error:  5.12370502948761
training error:  1.3050131797790527  and  2.1493589878082275  a

training error:  1.2382625341415405  and  1.842114806175232  and  1.4792684316635132
total error:  4.559645771980286
training error:  1.2185481786727905  and  1.8434884548187256  and  1.4447506666183472
total error:  4.506787300109863
training error:  1.2255820035934448  and  1.8285335302352905  and  1.4103959798812866
total error:  4.464511513710022
training error:  1.230910062789917  and  1.772304892539978  and  1.4131287336349487
total error:  4.416343688964844
training error:  1.1909915208816528  and  1.8601007461547852  and  1.2593872547149658
total error:  4.310479521751404
training error:  1.204993724822998  and  1.881202220916748  and  1.4790095090866089
total error:  4.565205454826355
training error:  1.2116484642028809  and  1.8271564245224  and  1.379004955291748
total error:  4.417809844017029
training error:  1.1799834966659546  and  1.8964927196502686  and  1.3821077346801758
total error:  4.458583950996399
training error:  1.1827647686004639  and  1.7544901371002197  and

training error:  1.1419776678085327  and  1.8048418760299683  and  1.3770833015441895
total error:  4.32390284538269
training error:  1.1107831001281738  and  1.8029401302337646  and  1.2556047439575195
total error:  4.169327974319458
training error:  1.1197806596755981  and  1.8206379413604736  and  1.3343799114227295
total error:  4.274798512458801
training error:  1.125493049621582  and  1.6406711339950562  and  1.253355860710144
total error:  4.019520044326782
training error:  1.1225330829620361  and  1.6401667594909668  and  1.2924052476882935
total error:  4.055105090141296
training error:  1.0880311727523804  and  1.7003262042999268  and  1.275097131729126
total error:  4.063454508781433
training error:  1.1091110706329346  and  1.6247878074645996  and  1.2767285108566284
total error:  4.010627388954163
training error:  1.1131985187530518  and  1.6765235662460327  and  1.2772393226623535
total error:  4.066961407661438
training error:  1.1287081241607666  and  1.8007919788360596

...

training error:  0.6454970836639404  and  1.1271156072616577  and  0.8973233103752136
total error:  2.6699360013008118
training error:  0.6250676512718201  and  1.1160988807678223  and  0.9026027917861938
total error:  2.643769323825836
training error:  0.6595950722694397  and  1.1290640830993652  and  0.9199907779693604
total error:  2.7086499333381653
training error:  0.638208270072937  and  1.0928668975830078  and  0.8591485023498535
total error:  2.5902236700057983
training error:  0.6459855437278748  and  1.1854891777038574  and  0.9941840767860413
total error:  2.8256587982177734
training error:  0.680299699306488  and  1.1986448764801025  and  0.9656565189361572
total error:  2.844601094722748
training error:  0.6327516436576843  and  1.1318528652191162  and  0.9232099056243896
total error:  2.68781441450119
training error:  0.6429406404495239  and  1.097536563873291  and  1.0304994583129883
total error:  2.7709766626358032
training error:  0.6334123611450195  and  1.19781851768

training error:  0.6673921346664429  and  1.1232575178146362  and  0.914074718952179
total error:  2.704724371433258
training error:  0.6227135062217712  and  1.1478989124298096  and  0.9519436359405518
total error:  2.7225560545921326
training error:  0.6590138673782349  and  1.1166348457336426  and  0.9409980177879333
total error:  2.716646730899811
training error:  0.6389837265014648  and  1.3119120597839355  and  1.2159051895141602
total error:  3.1668009757995605
training error:  0.6195387840270996  and  1.124267339706421  and  0.8793807029724121
total error:  2.6231868267059326
training error:  0.6435539722442627  and  1.1050307750701904  and  0.9356966614723206
total error:  2.6842814087867737
training error:  0.6181704998016357  and  1.1561086177825928  and  0.9581528306007385
total error:  2.732431948184967
training error:  0.6179817914962769  and  1.149308443069458  and  0.9177509546279907
total error:  2.6850411891937256
training error:  0.6433119773864746  and  1.1120676994

training error:  0.6174153685569763  and  1.1660767793655396  and  0.9799121618270874
total error:  2.7634043097496033
training error:  0.6102837324142456  and  1.1702117919921875  and  0.9301859140396118
total error:  2.710681438446045
training error:  0.6079088449478149  and  1.1288959980010986  and  0.905596911907196
total error:  2.6424017548561096
training error:  0.6355558037757874  and  1.15547513961792  and  1.1132344007492065
total error:  2.904265344142914
training error:  0.6277397871017456  and  1.1290069818496704  and  1.0089654922485352
total error:  2.765712261199951
training error:  0.6334760189056396  and  1.1263059377670288  and  0.9289933443069458
total error:  2.6887753009796143
training error:  0.6268795728683472  and  1.1003894805908203  and  1.1216068267822266
total error:  2.848875880241394
training error:  0.6709077954292297  and  1.1544378995895386  and  0.8812581300735474
total error:  2.7066038250923157
training error:  0.6322187781333923  and  1.06590938568

training error:  0.6060059666633606  and  1.0758438110351562  and  0.8649581074714661
total error:  2.546807885169983
training error:  0.6510473489761353  and  1.2356553077697754  and  0.8993418216705322
total error:  2.786044478416443
training error:  0.6787575483322144  and  1.3075759410858154  and  1.467695713043213
total error:  3.4540292024612427
training error:  0.6203290224075317  and  1.2382779121398926  and  0.9964420795440674
total error:  2.8550490140914917
training error:  0.6161158680915833  and  1.2696583271026611  and  0.8964725732803345
total error:  2.782246768474579
training error:  0.5876244902610779  and  1.054898738861084  and  0.8773679733276367
total error:  2.5198912024497986
training error:  0.5993084907531738  and  1.0615323781967163  and  0.8956220149993896
total error:  2.55646288394928
training error:  0.6003743410110474  and  1.0684823989868164  and  0.8658995032310486
total error:  2.5347562432289124
training error:  0.61202073097229  and  1.0654165744781

training error:  0.6298619508743286  and  1.3310742378234863  and  0.9133249521255493
total error:  2.8742611408233643
training error:  0.6826364398002625  and  1.1444909572601318  and  0.9376972913742065
total error:  2.764824688434601
training error:  0.6214983463287354  and  1.1100032329559326  and  0.9705526828765869
total error:  2.702054262161255
training error:  0.640586793422699  and  1.1488685607910156  and  1.0672321319580078
total error:  2.8566874861717224
training error:  0.610547661781311  and  1.2012039422988892  and  1.088070034980774
total error:  2.899821639060974
training error:  0.6679860353469849  and  1.1231969594955444  and  0.9144800901412964
total error:  2.7056630849838257
training error:  0.6284352540969849  and  1.136749267578125  and  0.953014612197876
total error:  2.718199133872986
training error:  0.7028155326843262  and  1.2173821926116943  and  0.9688256978988647
total error:  2.8890234231948853
training error:  0.630058765411377  and  1.12443947792053

In [2]:
torch.save(netG.state_dict(),"./epsilon0.1.pt")